# Web-scraping Wikipedia

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from datetime import datetime

In [2]:
url = "https://en.wikipedia.org/wiki/Berlin"

In [3]:
url1 = "https://en.wikipedia.org/wiki/Frankfurt"

In [6]:
url2 = "https://en.wikipedia.org/wiki/Munich"

In [9]:
response = requests.get(url)
#response.status_code # 200 status code means OK!

In [10]:
response1 = requests.get(url1)
response.status_code # 200 status code means OK!

200

In [11]:
response2 = requests.get(url2)
response.status_code # 200 status code means OK!

200

In [12]:
soup = BeautifulSoup(response.content, "html.parser")

In [13]:
soup1 = BeautifulSoup(response1.content, "html.parser")

In [14]:
soup2 = BeautifulSoup(response2.content, "html.parser")

In [15]:
#soup.prettify

In [16]:
city_names = []
country_names = []

## Search for city name

In [17]:
#<span class="mw-page-title-main">Berlin</span>

In [18]:
city = soup.select('span.mw-page-title-main')[0].get_text()
city_names.append(city)
city_names

['Berlin']

In [19]:
city1 = soup1.select('span.mw-page-title-main')[0].get_text()
city_names.append(city1)
city_names

['Berlin', 'Frankfurt']

In [20]:
soup2.select('span.mw-page-title-main')[0].get_text()

'Munich'

## Search for country

In [21]:
#<td class="infobox-data"><a href="/wiki/Germany" title="Germany">Germany</a></td>

In [22]:
country = soup.select('td.infobox-data')[0].get_text()
country_names.append(country)
country_names

['Germany']

In [23]:
country1 = soup1.select('td.infobox-data')[0].get_text()
country_names.append(country1)
country_names

['Germany', 'Germany']

In [24]:
country2 = soup2.select('td.infobox-data')[0].get_text()
country_names.append(country2)
country_names

['Germany', 'Germany', 'Germany']

## Search for latitude

In [25]:
#<span class="latitude">52°31′12″N</span>

In [26]:
soup.select('span.latitude')[0].get_text()

'52°31′12″N'

In [27]:
soup1.select('span.latitude')[0].get_text()

'50°06′38″N'

In [28]:
soup2.select('span.latitude')[0].get_text()

'48°08′15″N'

## Search for longitude

In [29]:
#<span class="longitude">13°24′18″E</span>

In [30]:
soup.select('span.longitude')[0].get_text()

'13°24′18″E'

In [31]:
soup1.select('span.longitude')[0].get_text()

'08°40′56″E'

In [32]:
soup2.select('span.longitude')[0].get_text()

'11°34′30″E'

## Search for population

In [33]:
#<td class="infobox-data">3,850,809</td>

In [34]:
soup.select('td.infobox-data')[10].get_text()

'3,850,809'

In [35]:
#Alternative Methode:

In [36]:
population = soup.select('th.infobox-header:-soup-contains("Population")')[0].parent.find_next_sibling().find(string=re.compile(r'\d+'))
population

'3,850,809'

In [76]:
soup1.select('div.ib-settlement-fn')

[<div class="ib-settlement-fn"></div>,
 <div class="ib-settlement-fn"></div>,
 <div class="ib-settlement-fn"></div>,
 <div class="ib-settlement-fn"><span class="nowrap"> </span>(2022-12-31)<sup class="reference" id="cite_ref-4"><a href="#cite_note-4">[4]</a></sup></div>,
 <div class="ib-settlement-fn"></div>,
 <div class="ib-settlement-fn"></div>]

## Search for Year of Poulation

In [77]:
#Get year of population
soup.select('div.ib-settlement-fn')

[<div class="ib-settlement-fn"></div>,
 <div class="ib-settlement-fn"></div>,
 <div class="ib-settlement-fn"></div>,
 <div class="ib-settlement-fn"><span class="nowrap"> </span>(2022-12-31)<sup class="reference" id="cite_ref-4"><a href="#cite_note-4">[4]</a></sup></div>,
 <div class="ib-settlement-fn"></div>,
 <div class="ib-settlement-fn"></div>]

In [78]:
divs = soup2.find_all('div', class_='ib-settlement-fn')

for div in divs:
    span = div.find('span')
    if span:
        year_text = span.next_sibling.strip('()')
year_text

'2022-12-31'

## Setting up loops to simplify the process

In [47]:
list_of_cities = ["Hamburg", "Berlin", "Frankfurt"]

for city in list_of_cities:

    url = "https://en.wikipedia.org/wiki/" + city
    headers = {'Accept-Language': 'en-US,en;q=0.8'}
    response = requests.get(url, headers = headers)
    if response.status_code != 200: break
    soup = BeautifulSoup(response.content, "html.parser")



city_names = []
country_names = []
latitudes = []
longitudes = []
populations = []

In [48]:
for city in list_of_cities:

    url = "https://en.wikipedia.org/wiki/" + city
    headers = {'Accept-Language': 'en-US,en;q=0.8'}
    response = requests.get(url, headers = headers)
    if response.status_code != 200: break
    soup = BeautifulSoup(response.content, "html.parser")
    
    city_name = soup.select('span.mw-page-title-main')[0].get_text()
    city_names.append(city_name)
    country_name = soup.select('td.infobox-data')[0].get_text()
    country_names.append(country_name)
    latitude = soup.select('span.latitude')[0].get_text()
    latitudes.append(latitude)
    longitude = soup.select('span.longitude')[0].get_text()
    longitudes.append(longitude)
    population = soup.select('th.infobox-header:-soup-contains("Population")')[0].parent.find_next_sibling().find(string=re.compile(r'\d+'))
    populations.append(population)

In [49]:
cities_df = pd.DataFrame(
    {"City": city_names,
     "Country": country_names,
     "Latitude": latitudes,
     "Longitude": longitudes,
     "Population": populations
    }
)

In [50]:
cities_df

,City,Country,Latitude,Longitude,Population
0,Hamburg,Germany,53°33′N,10°00′E,"1,945,532"
1,Berlin,Germany,52°31′12″N,13°24′18″E,"3,850,809"
2,Frankfurt,Germany,50°06′38″N,08°40′56″E,"773,068"


In [51]:
cities_df['Population'] = cities_df['Population'].str.replace(',', '').astype(int)

In [52]:
cities_df

,City,Country,Latitude,Longitude,Population
0,Hamburg,Germany,53°33′N,10°00′E,1945532
1,Berlin,Germany,52°31′12″N,13°24′18″E,3850809
2,Frankfurt,Germany,50°06′38″N,08°40′56″E,773068


In [53]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   City        3 non-null      object
 1   Country     3 non-null      object
 2   Latitude    3 non-null      object
 3   Longitude   3 non-null      object
 4   Population  3 non-null      int64 
dtypes: int64(1), object(4)
memory usage: 252.0+ bytes


# Final Way to set up the Dataframe

In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from datetime import datetime

In [8]:
list_of_cities = ['Berlin', 'Frankfurt', 'Munich', 'Cologne', 'Stuttgart', 'Leipzig']

In [9]:
def convert_latitude_longitude_to_decimal(input):
    try: # extract all 4 parts from the latitude / longitude string
        degrees, minutes, seconds, direction = re.split("[^\d\w]+", input)
    except: # exception for Hamburg (has only 3 parts)
        degrees, minutes, direction = re.split("[^\d\w]+", input)
        seconds = 0

    # transform parts into decimal
    decimal_degrees = int(degrees) + int(minutes)/60 + int(seconds)/(60*60)

    # change decimal to "-" if direction is South or West
    if (direction == "S" or direction == "W"):
        decimal_degrees *= -1

    return decimal_degrees

In [10]:
def get_city_data(list_of_cities):

    scooter_cities_df = pd.DataFrame(
            {"city": [],
             "country": [],
             "latitude": [],
             "longitude": [],
             "population": [],
             "timestamp": []
            })

    for city in list_of_cities:

        url = "https://en.wikipedia.org/wiki/" + city
        #download html with a get request
        headers = {'Accept-Language': 'en-US,en;q=0.8'}
        response = requests.get(url, headers = headers)
        if response.status_code != 200: break  # 200 status code means OK!

        # parse html (create the 'soup')
        wiki_soup = BeautifulSoup(response.content, "html.parser")

        #extract name, country, latitude, longitude, population
        city_name = wiki_soup.select("span.mw-page-title-main")[0].getText()
        country_name = wiki_soup.select("table.infobox td.infobox-data")[0].getText()
        latitude = wiki_soup.select("span.latitude")[0].getText()
        longitude = wiki_soup.select("span.longitude")[0].getText()
        
        divs = wiki_soup.find_all('div', class_='ib-settlement-fn')
        for div in divs:
            span = div.find('span')
            if span:
                year_text = span.next_sibling.strip('()')
              
        timestamp = year_text

        if wiki_soup.select_one('th.infobox-header:-soup-contains("Population")'):
            population = wiki_soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(string=re.compile(r'\d+'))


        # convert latitude, longitude and population to decimal (optional)
        latitude = convert_latitude_longitude_to_decimal(latitude)
        longitude = convert_latitude_longitude_to_decimal(longitude)
        population = int(population.replace(",", ""))


        #create a Dataframe from the new information
        city_df = pd.DataFrame(
            {"city": [city_name],
             "country": [country_name],
             "latitude": [latitude],
             "longitude": [longitude],
             "population": [population],
             "timestamp": [timestamp]
            }
        )
        # append info to scooter_cities_df
        scooter_cities_df = pd.concat([scooter_cities_df, city_df], ignore_index = True)

    return scooter_cities_df

In [11]:
cities_df = get_city_data(list_of_cities)
cities_df

,city,country,latitude,longitude,population,timestamp
0,Berlin,Germany,52.520000,13.405000,3850809.0,2021
1,Frankfurt,Germany,50.110556,8.682222,773068.0,2022-12-31
2,Munich,Germany,48.137500,11.575000,1512491.0,2022-12-31
3,Cologne,Germany,50.936389,6.952778,1073096.0,2021-12-31
4,Stuttgart,Germany,48.777500,9.180000,626275.0,2021-12-31
5,Leipzig,Germany,51.340000,12.375000,601866.0,2021-12-31


In [12]:
def clean_city_data(cities_df):
    
    cities_df['year_population'] = cities_df['timestamp'].apply(lambda x: re.search(r"\d{4}", x).group() if re.search(r"\d{4}", x) else None)
    cities_df.drop(columns='timestamp', inplace=True)
    
    return cities_df

In [13]:
cities_df_final = clean_city_data(cities_df)
cities_df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   city             6 non-null      object 
 1   country          6 non-null      object 
 2   latitude         6 non-null      float64
 3   longitude        6 non-null      float64
 4   population       6 non-null      float64
 5   year_population  6 non-null      object 
dtypes: float64(3), object(3)
memory usage: 420.0+ bytes


In [14]:
cities_df_final

,city,country,latitude,longitude,population,year_population
0,Berlin,Germany,52.520000,13.405000,3850809.0,2021
1,Frankfurt,Germany,50.110556,8.682222,773068.0,2022
2,Munich,Germany,48.137500,11.575000,1512491.0,2022
3,Cologne,Germany,50.936389,6.952778,1073096.0,2021
4,Stuttgart,Germany,48.777500,9.180000,626275.0,2021
5,Leipzig,Germany,51.340000,12.375000,601866.0,2021


# Exporting Data into MySQL Local

In [15]:
import sqlalchemy
from gans_con import con

In [95]:
cities_df_final.to_sql('cities',
              if_exists='append',
              con=con,
              index=False)

6

# Exporting Data into MySQL on the AWS Cloud server

In [23]:
import sqlalchemy
from gans_con_aws import con

In [24]:
cities_df_2 = cities_df_final.iloc[:2]

In [30]:
cities_df_2.to_sql('cities',
              if_exists='append',
              con=con,
              index=False)

2